# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [2]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [24]:
df = pd.read_csv("cities_data.csv")
df

,city,lat,long,max_temp,humidity,cloudiness,wind_spd,country,date
0,Timrå,62.49,17.33,276.48,93,0,3.10,SE,1604034070
1,Jiexiu,37.02,111.91,288.31,42,0,1.38,CN,1604034070
2,Esperance,-33.87,121.90,299.15,22,87,5.70,AU,1604034070
3,Bluff,-46.60,168.33,288.71,92,100,1.79,NZ,1604034070
4,Vaijāpur,19.92,74.73,299.73,40,0,1.47,IN,1604034070
...,...,...,...,...,...,...,...,...,...
547,Dingle,11.00,122.67,299.00,88,100,2.43,PH,1604034192
548,Soignies,50.58,4.07,286.48,87,97,8.70,BE,1604034192
549,Örnsköldsvik,63.29,18.72,274.15,100,0,2.10,SE,1604034192
550,Kasongo-Lunda,-6.47,16.82,294.70,93,100,0.71,CD,1604034192


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [25]:
gmaps.configure(api_key=g_key)

In [77]:
locations = df[["lat", "long"]]
humidity = df["humidity"]
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, dissipating=False, max_intensity=100, point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [38]:
#fahrenheit to kelvin
# 80 -> 299.817
# 70 -> 294.261
dff = df.loc[df["max_temp"] < 299.817,:]
dff = dff.loc[df["max_temp"] > 294.261,:]
dff = dff.loc[df["wind_spd"] < 10,:]
dff = dff.loc[df["cloudiness"] == 0,:]
dff

,city,lat,long,max_temp,humidity,cloudiness,wind_spd,country,date
4,Vaijāpur,19.92,74.73,299.73,40,0,1.47,IN,1604034070
8,Sapouy,11.55,-1.77,296.03,29,0,1.20,BF,1604034071
31,Tessalit,20.20,1.01,295.33,20,0,3.28,ML,1604034076
53,Carnarvon,-24.87,113.63,299.15,65,0,7.70,AU,1604034082
73,Bam,29.11,58.36,295.23,19,0,2.30,IR,1604034087
111,Dhārchula,29.85,80.53,296.34,19,0,2.24,IN,1604034096
160,Salalah,17.02,54.09,299.15,65,0,0.50,OM,1604034107
172,Diffa,13.32,12.61,298.43,25,0,4.15,NE,1604034110
186,Marzuq,14.40,46.47,294.76,23,0,1.16,YE,1604034113
214,Djibo,14.10,-1.63,298.63,19,0,2.49,BF,1604034120


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [122]:
hotel_df = dff.copy()
hotel_df["Hotel Name"] = " "
hotel_df

,city,lat,long,max_temp,humidity,cloudiness,wind_spd,country,date,Hotel Name
4,Vaijāpur,19.92,74.73,299.73,40,0,1.47,IN,1604034070,
8,Sapouy,11.55,-1.77,296.03,29,0,1.20,BF,1604034071,
31,Tessalit,20.20,1.01,295.33,20,0,3.28,ML,1604034076,
53,Carnarvon,-24.87,113.63,299.15,65,0,7.70,AU,1604034082,
73,Bam,29.11,58.36,295.23,19,0,2.30,IR,1604034087,
111,Dhārchula,29.85,80.53,296.34,19,0,2.24,IN,1604034096,
160,Salalah,17.02,54.09,299.15,65,0,0.50,OM,1604034107,
172,Diffa,13.32,12.61,298.43,25,0,4.15,NE,1604034110,
186,Marzuq,14.40,46.47,294.76,23,0,1.16,YE,1604034113,
214,Djibo,14.10,-1.63,298.63,19,0,2.49,BF,1604034120,


In [127]:
for city in hotel_df["city"]:
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    coords = [(hotel_df.loc[city == hotel_df["city"],"lat"]).item(),(hotel_df.loc[city == hotel_df["city"],"long"]).item()]
    params = {
        "key": g_key,
        "location": f"{coords[0]},{coords[1]}",
        "radius": 5000,
        "type": "lodging"
    }
    request = requests.get(base_url, params=params).json()
    request
    try:
        hotel_df.loc[city == hotel_df["city"], "Hotel Name"] = request["results"][0]["name"]
    except (KeyError, IndexError):
        print("Hotel not found")
        #hotel_df = hotel_df.drop(hotel_df.loc[city == hotel_df["city"],:])
hotel_df

Hotel not found
Hotel not found
Hotel not found


,city,lat,long,max_temp,humidity,cloudiness,wind_spd,country,date,Hotel Name
4,Vaijāpur,19.92,74.73,299.73,40,0,1.47,IN,1604034070,HOTEL MATOSHRI
8,Sapouy,11.55,-1.77,296.03,29,0,1.20,BF,1604034071,Auberge St Louis
31,Tessalit,20.20,1.01,295.33,20,0,3.28,ML,1604034076,
53,Carnarvon,-24.87,113.63,299.15,65,0,7.70,AU,1604034082,Hospitality Carnarvon
73,Bam,29.11,58.36,295.23,19,0,2.30,IR,1604034087,Bam Tourist Hotel
111,Dhārchula,29.85,80.53,296.34,19,0,2.24,IN,1604034096,Mahara Hotel
160,Salalah,17.02,54.09,299.15,65,0,0.50,OM,1604034107,HAMDAN PLAZA HOTEL
172,Diffa,13.32,12.61,298.43,25,0,4.15,NE,1604034110,ProDAF/Diffa
186,Marzuq,14.40,46.47,294.76,23,0,1.16,YE,1604034113,
214,Djibo,14.10,-1.63,298.63,19,0,2.49,BF,1604034120,Centre Des Ressources DPENA


In [129]:
hotel_df = hotel_df.drop([31,186,397])

In [130]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["lat", "long"]]

KeyError: 'City'

In [ ]:
# Add marker layer ontop of heat map


# Display figure
markers = gmaps.marker_layer(coordinates)
# Add the layer to the map
fig.add_layer(markers)
fig
# ignore the bottom mess

In [117]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
coords = [(hotel_df.loc["Bam" == hotel_df["city"],"lat"]).item(),(hotel_df.loc["Bam" == hotel_df["city"],"long"]).item()]
params = {
    "key": g_key,
    "location": f"{coords[0]},{coords[1]}",
    "radius": 5000,
    "type": "lodging"
}
request = requests.get(base_url, params=params).json()
request

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 29.095207, 'lng': 58.3566841},
    'viewport': {'northeast': {'lat': 29.0965929802915,
      'lng': 58.3578645802915},
     'southwest': {'lat': 29.0938950197085, 'lng': 58.35516661970849}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Bam Tourist Hotel',
   'photos': [{'height': 640,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110696951891970643324">Rezvan Pap</a>'],
     'photo_reference': 'CmRaAAAABsHTj9EO4LwP9qNhLkcnBz7gADpAUAvW0h18nzB0IF7LuuARxJ__yjcaRyPDD84lSezl_Sz2Fx8akjKOtwXt15_u1dJ4t0RbhSnEy3peG01l_R6n_Rg1rwHmhjmSMoecEhAP6XvpMJvOZ3lbtidMGzfZGhS0dhVKvkaVvX-FGKUUenYguKgr1g',
     'width': 640}],
   'place_id': 'ChIJScE2ul6r_D4RXasbBFYwHxY',
   'plus_code': {'compound_code': '39W4+3M Bam, Kerman Province, Iran',
    'global_code': '7HXW39W4+3M'},
   'rating': 3.7,
   'reference': 'ChIJS

In [114]:
(hotel_df.loc["Bam" == hotel_df["city"],"lat"]).item()
coords

[29.11, 58.36]

In [115]:
params = {
    "key": g_key,
    "location": f"{coords[0]},{coords[1]}",
    "radius": 5000,
    "type": "lodging"
}
request = requests.get(base_url, params=params).json()
request

{'html_attributions': [],
 'results': [{'business_status': 'OPERATIONAL',
   'geometry': {'location': {'lat': 29.095207, 'lng': 58.3566841},
    'viewport': {'northeast': {'lat': 29.0965929802915,
      'lng': 58.3578645802915},
     'southwest': {'lat': 29.0938950197085, 'lng': 58.35516661970849}}},
   'icon': 'https://maps.gstatic.com/mapfiles/place_api/icons/v1/png_71/lodging-71.png',
   'name': 'Bam Tourist Hotel',
   'photos': [{'height': 640,
     'html_attributions': ['<a href="https://maps.google.com/maps/contrib/110696951891970643324">Rezvan Pap</a>'],
     'photo_reference': 'CmRaAAAAvf6vAlxvNcxM7F29Aew1W74-lpzr7u-g0vKCUzXB_z3oycVS00dVuP61NcbfQ8y_7tQo8Bayej0ZmJoIe1qNPKVh_5-q4-0AJziV_Gl6GeUvB4YQo8ibZqE4fkxNDLlyEhAd45SFLyLV5p-iQbUqncciGhSgn2adU45uReQBPCD4lA4Ab0FAaQ',
     'width': 640}],
   'place_id': 'ChIJScE2ul6r_D4RXasbBFYwHxY',
   'plus_code': {'compound_code': '39W4+3M Bam, Kerman Province, Iran',
    'global_code': '7HXW39W4+3M'},
   'rating': 3.7,
   'reference': 'ChIJS